# Constraint statistics
Explore trends of regional nonsense constraint. 

In [1]:
import pandas as pd
from scipy import stats

In [2]:
# Load data
df = pd.read_csv("data/final/regional_nonsense_constraint.tsv", sep="\t")
df.sample(2)

,enst,region,csq,n_pos,n_obs,n_exp,oe,prop_obs,prop_exp,mu,p,oe_ci_hi,fdr_p,pli,loeuf,gnomad_flags,syn_p,constraint
27708,ENST00000445101,start_proximal,stop_gained,18,4,3.229135,1.238722,0.222222,0.179396,1.820873e-08,0.790424,2.446441,0.920216,2.419800e-01,1.759,[],0.954021,unconstrained
31525,ENST00000227880,nmd_target,stop_gained,126,24,26.441326,0.907670,0.190476,0.209852,2.129996e-08,0.342038,1.226042,0.651632,8.926600e-13,1.191,[],0.295002,unconstrained


## Number of constrained regions

In [3]:
# Masks for constraint criteria
m1 = df.syn_p >= stats.norm.cdf(-1)
m3 = df.fdr_p < 0.05
m4 = df.oe_ci_hi <= 0.6
m5 = df.n_obs == 0

# Cumulative criteria
ma = (m1).rename("pass_syn_p")
mb = (ma & m3).rename("pass_fdr")
mc = (mb & m4).rename("pass_oe_ci_hi")
md = (mc & m5).rename("none_observed")

df = pd.concat([df, ma, mb, mc, md], axis=1)

In [4]:
# Regions meeting cumulative criteria
valid = df.groupby("region").agg(
    transcripts=("enst","nunique"),
    valid_statistics=("p", "count"),
    pass_syn_p=("pass_syn_p", "sum"),
    pass_fdr=("pass_fdr", "sum"),
    pass_oe_ci_hi=("pass_oe_ci_hi", "sum"),
    none_observed=("none_observed", "sum"),
)

valid

,transcripts,valid_statistics,pass_syn_p,pass_fdr,pass_oe_ci_hi,none_observed
region,,,,,,
distal_nmd,18192,18189,15014,1023,555,190
long_exon,3315,3315,2740,596,374,59
nmd_target,15878,15878,12352,3500,1912,141
start_proximal,18272,18270,15645,127,92,89
transcript,18647,18647,14191,5518,2347,98
